# STATE SPACE MODEL MATRICES

### Diff drive

In [1]:
import sympy as sp

x,y,theta,psi,cte,v,w = sp.symbols("x y theta psi cte v w")

gs = sp.Matrix([[ sp.cos(theta)*v],
               [ sp.sin(theta)*v],
               [w],
               [-w],
               [ v*sp.sin(psi)]])

state = sp.Matrix([x,y,theta,psi,cte])

#A
gs.jacobian(state)

Matrix([
[0, 0, -v*sin(theta),          0, 0],
[0, 0,  v*cos(theta),          0, 0],
[0, 0,             0,          0, 0],
[0, 0,             0,          0, 0],
[0, 0,             0, v*cos(psi), 0]])

In [2]:
state = sp.Matrix([v,w])

#B
gs.jacobian(state)

Matrix([
[cos(theta),  0],
[sin(theta),  0],
[         0,  1],
[         0, -1],
[  sin(psi),  0]])

In [3]:
import sympy as sp

x,y,theta,psi,cte,v,w ,dt= sp.symbols("x y theta psi cte v w dt")

gs = sp.Matrix([[x + sp.cos(theta)*v*dt],
               [y+ sp.sin(theta)*v*dt],
               [theta + w*dt]])

state = sp.Matrix([x,y,theta])

#A
gs.jacobian(state)#.subs({x:0,y:0,theta:0})

Matrix([
[1, 0, -dt*v*sin(theta)],
[0, 1,  dt*v*cos(theta)],
[0, 0,                1]])

In [4]:
state = sp.Matrix([v,w])

#B
gs.jacobian(state)#.subs({x:0,y:0,theta:0})

Matrix([
[dt*cos(theta),  0],
[dt*sin(theta),  0],
[            0, dt]])

## Ackermann Kinematics model

### Jacobians

In [5]:
x,y,theta,v,delta,L,a = sp.symbols("x y theta v delta L a")

gs = sp.Matrix([[ sp.cos(theta)*v],
               [ sp.sin(theta)*v],
               [a],
               [ v*sp.tan(delta)/L]])

X = sp.Matrix([x,y,v,theta])

#A
A=gs.jacobian(X)

U = sp.Matrix([a,delta])

#B
B=gs.jacobian(U)
display(A)
display(B)

Matrix([
[0, 0,   cos(theta), -v*sin(theta)],
[0, 0,   sin(theta),  v*cos(theta)],
[0, 0,            0,             0],
[0, 0, tan(delta)/L,             0]])

Matrix([
[0,                       0],
[0,                       0],
[1,                       0],
[0, v*(tan(delta)**2 + 1)/L]])

### Discretized and Linearized model

In [9]:
DT = sp.symbols("dt")

display(sp.eye(4)+A*DT)
display(B*DT)
display(DT*(gs - A*X - B*U))

Matrix([
[1, 0,   dt*cos(theta), -dt*v*sin(theta)],
[0, 1,   dt*sin(theta),  dt*v*cos(theta)],
[0, 0,               1,                0],
[0, 0, dt*tan(delta)/L,                1]])

Matrix([
[ 0,                          0],
[ 0,                          0],
[dt,                          0],
[ 0, dt*v*(tan(delta)**2 + 1)/L]])

Matrix([
[            dt*theta*v*sin(theta)],
[           -dt*theta*v*cos(theta)],
[                                0],
[-delta*dt*v*(tan(delta)**2 + 1)/L]])

# ADD DELAY (for real time implementation)

It is necessary to take *actuation latency* into account: so instead of using the actual state as estimated, the delay factored in using the kinematic model

Starting State is :

* $x_{delay} = 0.0 + v * dt$
* $y_{delay} = 0.0$
* $psi_{delay} = 0.0 + w * dt$
* $cte_{delay} = cte + v * sin(epsi) * dt$
* $epsi_{delay} = epsi - w * dt$

Note that the starting position and heading is always 0; this is becouse the path is parametrized to **vehicle reference frame**